# Lab 3: Regularized Regression Pipeline

Dataset: Ames Housing (House Prices)

### Goal: Predict the SalePrice of a home using physical attributes while preventing overfitting through regularization.

### 1. Objective

Build an end-to-end regression pipeline. Make sure your model handles numerical data, encode neighborhood information, and use Cross-Validation to decide which regularization method (Lasso or Ridge) is better for predicting home values. Compare it against a non-regularized model, what performs better?

### 2. Key Concepts and Terms

**High Cardinality**: The `Neighborhood` column has many unique values. We must encode this carefully.

**The Alpha ($\alpha$) Parameter**: In Scikit-Learn's Ridge and Lasso, $\alpha$ is the regularization strength.

$\alpha = 0$: Standard Linear Regression.

$\alpha > 0$: Increasing penalty on model complexity.

**Data Leakage:** A critical error where information from the test set (like the mean house price) "leaks" into the training process.

In [1]:
#Necessary imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge, Lasso
from sklearn.datasets import fetch_openml

**Task 1: Data Preparation**

Load the dataset and separate the target (`SalePrice`) from the features. Perform an 80/20 split.

In [ ]:
housing = fetch_openml(name="house_prices", as_frame=True, parser='auto')
X = housing.data[['GrLivArea', 'LotArea', 'OverallQual', 'Neighborhood', 'HouseStyle']]
y = housing.target

# TODO: Split the data (80/20 split, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # View my training data
# print(X_train.head())
# print(y_train.head())

      GrLivArea  LotArea  OverallQual Neighborhood HouseStyle
254        1314     8400            5        NAmes     1Story
1066       1571     7837            6      Gilbert     2Story
638         796     8777            5      Edwards     1Story
799        1768     7200            5        SWISU     1.5Fin
380        1691     5000            5        SWISU     1.5Fin
254     145000
1066    178000
638      85000
799     175000
380     127000
Name: SalePrice, dtype: int64


**Task 2: The Preprocessing Recipe**

Create a `ColumnTransformer` that applies different logic to different columns:

Numeric Features (`GrLivArea`, `LotArea`): These have huge scales. Apply `StandardScaler`.

Categorical Features (`Neighborhood`, `HouseStyle`): Use `OneHotEncoder`. Use `handle_unknown='ignore'` because a test set might have a neighborhood the training set didn't see.

In [18]:
numeric_features = ['GrLivArea', 'LotArea', 'OverallQual']
categorical_features = ['Neighborhood', 'HouseStyle']

# TODO: Create a numeric transformer (Imputer + Scaler)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# TODO: Create a categorical transformer (Imputer + OneHotEncoder)
# Hint: remember drop='first' to avoid the dummy variable trap!
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

**Task 3: Pipeline**

Construct a Pipeline that connects your **preprocessor** to a regressor. You will test both **Ridge()** and **Lasso().**

In [19]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# TODO: Create a pipeline that joins the preprocessor with Ridge regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

**Task 4: Hyperparameter Tuning**

Use `GridSearchCV` to find the best `alpha` for your model. Test values across several orders of magnitude (e.g., 0.1, 1, 10, 100).

In [20]:
param_grid = {
    # YOUR CODE HERE
    'regressor__alpha': [0.1, 1.0, 10.0, 100.0]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

/Users/adamzhang/Projects/ML@P/.venv/lib/python3.14/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/Users/adamzhang/Projects/ML@P/.venv/lib/python3.14/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/Users/adamzhang/Projects/ML@P/.venv/lib/python3.14/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/Users/adamzhang/Projects/ML@P/.venv/lib/python3.14/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories wil

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","Pipeline(step...r', Ridge())])"
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'regressor__alpha': [0.1, 1.0, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'r2'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and can

**Task 5: Evaluation**

Print out the best params for `alpha` and the corresponding R^2 score.

In [21]:
print(f"Best Alpha: {grid_search.best_params_}")
print(f"Test R^2 Score: {grid_search.score(X_test, y_test):.4f}")

Best Alpha: {'regressor__alpha': 1.0}
Test R^2 Score: 0.8246


3. **Questions to Consider**



*   Compare the results if we change the regressor to `Lasso()`, Does the R^@ score improve?
*   Lasso can set coefficients to zero, why might that be good for datasets like this with 70+ features?
*   If we ended up not using `Pipeline` and scaling our data before the train-test split, what implications does this have on the model's accuracy score?



In [ ]:
# ! Now let's do the same thing but with Lasso regression instead of Ridge regression.

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# TODO: Create a pipeline that joins the preprocessor with Lasso regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso())
])

param_grid = {
    # YOUR CODE HERE
    'regressor__alpha': [0.1, 1.0, 10.0, 100.0]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

print(f"Best Alpha: {grid_search.best_params_}")
print(f"Test R^2 Score: {grid_search.score(X_test, y_test):.4f}")

# ?   Compare the results if we change the regressor to `Lasso()`, Does the R^@ score improve?
    # ^ The R^2 score did not improve, it actually got worse when switching to Lasso regression. This is likely because Lasso regression 
    # ^ can set coefficients to zero, which may not be ideal for this dataset with 70+ features.
    
# ?   Lasso can set coefficients to zero, why might that be good for datasets like this with 70+ features?
    # ^ Lasso regression can perform feature selection by setting some coefficients to zero, 
    # ^ which can help reduce overfitting and improve model interpretability when dealing with datasets that have a large number of features.

# ?   If we ended up not using `Pipeline` and scaling our data before the train-test split, what implications does this have on the model's accuracy score?
    # ^ If we scale the data before the train-test split, we would be using information from the test set to inform the scaling of the training set
    # ^ , which can lead to data leakage and an overly optimistic accuracy score. 
    # ^ This is because the model would have access to information about the distribution of the test set during training, which it should not have.

Best Alpha: {'regressor__alpha': 10.0}
Test R^2 Score: 0.8244


/Users/adamzhang/Projects/ML@P/.venv/lib/python3.14/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/Users/adamzhang/Projects/ML@P/.venv/lib/python3.14/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/Users/adamzhang/Projects/ML@P/.venv/lib/python3.14/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/Users/adamzhang/Projects/ML@P/.venv/lib/python3.14/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories wil